In [1]:
%load_ext autoreload
%autoreload 3

import random

from pyfrost import frost

from bitcoinutils.setup import setup
from bitcoinutils.schnorr import int_from_bytes, bytes_from_int
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.keys import PrivateKey, P2wpkhAddress
from bitcoinutils.constants import TAPROOT_SIGHASH_ALL

DEBUG = True

install `bitcoinutils` before running this notebook:
```bash
pip install bitcoin-utils
```
Note:
The transaction which will be signed with this notebook actually exists on the bitcoin mainnet with txid: `bdf2f23bb2913d2ad427abd2cd512098339a329ee0e1ec42c6d07b9384aca43c`


# Generate distributed private key

In [2]:
DKG_ID = "0"
N = 3
T = 2
PARTY = ["1","2","3"]
COEF0 = [
    76622783108274780747756398270842445905196011304614807988789919767093500641526,
    31539798353799524717306909827363202534126249987352659471385619342384101334542,
    11876258468244277196215122101097336748296574692628866445318678405621527234717,
]
master_secret = 4246750693002387237707445190615077334781271705521429522889054373580967716448
DKG_ID, PARTY, COEF0

('0',
 ['1', '2', '3'],
 [76622783108274780747756398270842445905196011304614807988789919767093500641526,
  31539798353799524717306909827363202534126249987352659471385619342384101334542,
  11876258468244277196215122101097336748296574692628866445318678405621527234717])

In [3]:
setup("mainnet")
priv = PrivateKey(secret_exponent=master_secret)
pub = priv.get_public_key()
print(pub.to_hex())
print(pub.get_taproot_address().to_string())

0290ab36a28c3107d0091e8ea731e3c528046e317f854ed7b35f746d4e3484b1b3
bc1p48a8x62eny909e65mvwvvwgjyz84ggzg4m6uhq53sjjxx0a7expsfm2azt


In [4]:
fromAddress = pub.get_taproot_address()
# UTXO of fromAddress
txid = "be8c30cb34c4783750c967cb4954e92a111e70710b06accbb24957e26a815836"
vout = 1

# all amounts are needed to sign a taproot input
# (depending on sighash)

first_amount = 5700
amounts = [first_amount]

first_script_pubkey = fromAddress.to_script_pub_key()

utxos_script_pubkeys = [first_script_pubkey]
toAddress = P2wpkhAddress("bc1qst5cxg66xylpwx2hdssrrmwwqezgd50j2c3704")

# create transaction input from tx id of UTXO
txin = TxInput(txid, vout)

# create transaction output
txOut = TxOutput(first_amount - 2000, toAddress.to_script_pub_key())

# create transaction without change output - if at least a single input is
# segwit we need to set has_segwit=True
tx = Transaction([txin], [txOut], has_segwit=True)
tx_digest = tx.get_transaction_taproot_digest(
    0, utxos_script_pubkeys, amounts, 0, sighash=TAPROOT_SIGHASH_ALL
)


In [5]:
keys: list[frost.KeyGen] = []
sign_keys: list[frost.Key] = []
saved_data = {}

for coef0, node_id in zip(COEF0, PARTY):
    partners = PARTY.copy()
    partners.remove(node_id)
    keys.append(frost.KeyGen(DKG_ID, T, N, node_id, partners, coef0))

round1_received_data = []
for key in keys:
    round1_send_data = key.round1()
    round1_received_data.append(round1_send_data)
round2_received_data = {}
for node_id in PARTY:
    round2_received_data[node_id] = []
for key in keys:
    round2_send_data = key.round2(round1_received_data)
    for message in round2_send_data:
        round2_received_data[message["receiver_id"]].append(message)
dkg_keys = set()
for key in keys:
    result = key.round3(round2_received_data[key.node_id])
    dkg_keys.add(result["data"]["dkg_public_key"])
    sign_keys.append(frost.Key(key.dkg_key_pair, key.node_id))

saved_data['common_data'] = {}
saved_data['private_data'] = {}
for key in keys:
    nonces_common_data, nonces_private_data = frost.create_nonces(
        int(key.node_id), 10)
    saved_data['common_data'].update({key.node_id: nonces_common_data})
    saved_data['private_data'].update(
        {key.node_id: {'nonces': nonces_private_data}})
[s.dkg_key_pair for s in sign_keys]

[{'share': 260214056083543050521976932072618425897411474456882828841197837206325593718290,
  'dkg_public_key': X: 0x90ab36a28c3107d0091e8ea731e3c528046e317f854ed7b35f746d4e3484b1b3
  Y: 0x9bf1e979823f8b2fa05302ff90d622510bef9d3033eae6a486a0387387ff7ac2
  (On curve <secp256k1>)},
 {'share': 400389272236767518382675433945933866607204112929169323776901456897552058225795,
  'dkg_public_key': X: 0x90ab36a28c3107d0091e8ea731e3c528046e317f854ed7b35f746d4e3484b1b3
  Y: 0x9bf1e979823f8b2fa05302ff90d622510bef9d3033eae6a486a0387387ff7ac2
  (On curve <secp256k1>)},
 {'share': 540564488389991986243373935819249307316996751401455818712605076588778522733300,
  'dkg_public_key': X: 0x90ab36a28c3107d0091e8ea731e3c528046e317f854ed7b35f746d4e3484b1b3
  Y: 0x9bf1e979823f8b2fa05302ff90d622510bef9d3033eae6a486a0387387ff7ac2
  (On curve <secp256k1>)}]

In [6]:
sign_subset = random.sample(sign_keys, T)
commitments_data = {}
for key in sign_subset:
    # commitment = saved_data['common_data'][key.node_id].pop()
    commitment = saved_data['common_data'][key.node_id][0]
    commitments_data[key.node_id] = commitment
commitments_data

{'3': {'id': 3,
  'public_nonce_d': 427163159988314646247584188695683113472570043255019202432045418234711202560580,
  'public_nonce_e': 255945992013064246405232249967243571120479311278776310973376844770896831622443},
 '2': {'id': 2,
  'public_nonce_d': 234653528736208866855486639944222345826252833399106804939181258974416333461557,
  'public_nonce_e': 460370633238600515542057318689510794896448738961552678804243219294466639477316}}

In [7]:
signs = []
agregated_nonces = []
# msg = b'o\xde\xdd\xc6\xb1!\xdc<\xa9\x08]\t\xa1>p\xe2!\xcfr\x9e<\x87T\xce\xc5;\r\xa4p (\xc7'

for key in sign_subset:
    print("start")

    # single_sign, remove_data = key.sign(
    #     commitments_data, msg, saved_data["private_data"][key.node_id]["nonces"]
    # )

    single_sign, remove_data = key.sign(
        commitments_data,
        tx_digest,
        saved_data["private_data"][key.node_id]["nonces"],
    )
    # saved_data["private_data"][key.node_id]["nonces"].remove(remove_data)
    agregated_nonces.append(single_sign["aggregated_public_nonce"])

    signs.append(single_sign)

commitments_data, agregated_nonces, signs, sign_subset[0].dkg_key_pair

start
start


({'3': {'id': 3,
   'public_nonce_d': 427163159988314646247584188695683113472570043255019202432045418234711202560580,
   'public_nonce_e': 255945992013064246405232249967243571120479311278776310973376844770896831622443},
  '2': {'id': 2,
   'public_nonce_d': 234653528736208866855486639944222345826252833399106804939181258974416333461557,
   'public_nonce_e': 460370633238600515542057318689510794896448738961552678804243219294466639477316}},
 [X: 0xd8e06e9ba147639b9d4d5993a39bf0cf559a7b52a4085aabec3f5fe4e9c95fff
  Y: 0x9c1c2c073282721cf40769e6e0bb49c388bdf744ceb222cb3e2612ed09d86607
  (On curve <secp256k1>),
  X: 0xd8e06e9ba147639b9d4d5993a39bf0cf559a7b52a4085aabec3f5fe4e9c95fff
  Y: 0x9c1c2c073282721cf40769e6e0bb49c388bdf744ceb222cb3e2612ed09d86607
  (On curve <secp256k1>)],
 [{'id': 3,
   'signature': 56178521290608126376077952187773679324270379982408481561033050602953740872912,
   'public_key': 370118724089504200124903216784200312525825085195558868135484683967972212696371,
   'aggregated

In [8]:
group_sign = frost.aggregate_signatures(
    tx_digest,
    signs,
    agregated_nonces[0],
    result["data"]["dkg_public_key"],
)
group_sign


{'public_nonce': X: 0xd8e06e9ba147639b9d4d5993a39bf0cf559a7b52a4085aabec3f5fe4e9c95fff
 Y: 0x9c1c2c073282721cf40769e6e0bb49c388bdf744ceb222cb3e2612ed09d86607
 (On curve <secp256k1>),
 'public_key': X: 0x90ab36a28c3107d0091e8ea731e3c528046e317f854ed7b35f746d4e3484b1b3
 Y: 0x9bf1e979823f8b2fa05302ff90d622510bef9d3033eae6a486a0387387ff7ac2
 (On curve <secp256k1>),
 'signature': 97648874783380722786366259769237333702373057583879030507924491006908183570453,
 'message_hash': b'2\xa4u\x9d\x83\xfd{\x91E\xa6G\xc7\xa1\x94,-G\xd9\x8e\xc3}\x91{\xaa\xde\x1c\x80f\x1d\xa5\xd9N'}

In [9]:
## sign transaction using the master key
setup("mainnet")
priv = PrivateKey(secret_exponent=master_secret)
pub = priv.get_public_key()
display(pub.to_hex())
display(pub.get_taproot_address().to_string())

fromAddress = pub.get_taproot_address()
print(fromAddress.to_string())
# UTXO of fromAddress
txid = "be8c30cb34c4783750c967cb4954e92a111e70710b06accbb24957e26a815836"
vout = 1

# all amounts are needed to sign a taproot input
# (depending on sighash)

first_amount = 5700
amounts = [first_amount]

first_script_pubkey = fromAddress.to_script_pub_key()

utxos_script_pubkeys = [first_script_pubkey]
toAddress = P2wpkhAddress("bc1qst5cxg66xylpwx2hdssrrmwwqezgd50j2c3704")

# create transaction input from tx id of UTXO
txin = TxInput(txid, vout)

# create transaction output
txOut = TxOutput(first_amount - 2000, toAddress.to_script_pub_key())

# create transaction without change output - if at least a single input is
# segwit we need to set has_segwit=True
tx = Transaction([txin], [txOut], has_segwit=True)
tx_digest = tx.get_transaction_taproot_digest(
    0, utxos_script_pubkeys, amounts, 0, sighash=TAPROOT_SIGHASH_ALL
)
print("\ntx digest:\n" + str(tx_digest))
print("\ntx digest hex:\n" + tx_digest.hex())

print("\ntxid: " + tx.get_txid())
print("\ntxwid: " + tx.get_wtxid())


sig = bytes_from_int(group_sign["public_nonce"].x) + bytes_from_int(
    group_sign["signature"]
)
print("signature:", sig.hex())
print("R:", int_from_bytes(sig[:32]))
print("s:", int_from_bytes(sig[32:]))
# sig = b'%\x15\xf2\x84\x0f\x1d\x95\x104KK\x00:}\xe5w!\xf3\xe4&\xa6\xa6\xe3\xa6\xd3i<\xce\x9d\xa1\xb9\xf4dj\x95\xa7\x1f\xc5\xcb\x1c\x8b\x8c\x866\xe9\xba\x1e\xfbt\xaa\xad\x97\x1c$f\xf0\xa72yLVn\x01P'

tx.witnesses.append(TxWitnessInput([sig.hex()]))

# print raw signed transaction ready to be broadcasted
print("\nRaw signed transaction:\n" + tx.serialize())

print("\nTxId:", tx.get_txid())
print("\nTxwId:", tx.get_wtxid())

print("\nSize:", tx.get_size())
print("\nvSize:", tx.get_vsize())

'0290ab36a28c3107d0091e8ea731e3c528046e317f854ed7b35f746d4e3484b1b3'

'bc1p48a8x62eny909e65mvwvvwgjyz84ggzg4m6uhq53sjjxx0a7expsfm2azt'

bc1p48a8x62eny909e65mvwvvwgjyz84ggzg4m6uhq53sjjxx0a7expsfm2azt

tx digest:
b'2\xa4u\x9d\x83\xfd{\x91E\xa6G\xc7\xa1\x94,-G\xd9\x8e\xc3}\x91{\xaa\xde\x1c\x80f\x1d\xa5\xd9N'

tx digest hex:
32a4759d83fd7b9145a647c7a1942c2d47d98ec37d917baade1c80661da5d94e

txid: bdf2f23bb2913d2ad427abd2cd512098339a329ee0e1ec42c6d07b9384aca43c

txwid: af99e53cbc8a513aabb5659be4332edc0d3f86cd13c547fc910fa7789e27f7c9
signature: d8e06e9ba147639b9d4d5993a39bf0cf559a7b52a4085aabec3f5fe4e9c95fffd7e34df58aceae3bddb3ebee368f02159f14674c49e52539d8d11379c25b6815
R: 98096112424370425077034809776438493794353017702109009499814790104205744496639
s: 97648874783380722786366259769237333702373057583879030507924491006908183570453

Raw signed transaction:
020000000001013658816ae25749b2cbac060b71701e112ae95449cb67c9503778c434cb308cbe0100000000ffffffff01740e00000000000016001482e983235a313e1719576c2031edce064486d1f20140d8e06e9ba147639b9d4d5993a39bf0cf559a7b52a4085aabec3f5fe4e9c95fffd7e34df58aceae3bddb3ebee368f02159f14674c49e52539